In [1]:
import pandas as pd
import numpy as np
import feather
import pvlib
import sys
sys.path.append('..')
from src.utils.download_data import timer

In [2]:
def clear_sky(df, info=None, model='ineichen'):
    ''' Compute the total direct radiation under a clear sky model.
    '''
    lat, lon = info.loc[df.name, ['Latitude', 'Longitude']]
    location = pvlib.location.Location(lat, lon, tz='Pacific/Honolulu')
    return(location.get_clearsky(df.index.get_level_values('Datetime'), model=model))

In [3]:
# read minute data and location info
df = (pd.read_pickle('/home/SHARED/SOLAR/data/oahu_min_norm.pkl')
        .set_index(['Datetime', 'Location']))

info = pd.read_csv('/home/SHARED/SOLAR/data/info.csv')

# normalize location names by removing "HL_"
info['Location'] = info['Location'].str.replace('(HL)?_', '')
info.set_index('Location', inplace=True)

In [4]:
df.head()

Radiation                                GH       GT    ClearSky
Datetime                  Location                              
2010-03-19 14:15:00-10:00 AP1       476.328      NaN  973.629240
                          AP3       382.777      NaN  973.625962
                          AP4       351.610      NaN  973.624679
                          AP5       390.092      NaN  973.630994
                          AP6       353.928  343.313  973.628714

In [5]:
with timer():
    CS = df.groupby('Location').apply(clear_sky, info=info, model='ineichen')

Elapsed time (s): 161.101768


In [6]:
with timer():
    HCS = df.groupby('Location').apply(clear_sky, info=info, model='haurwitz')

Elapsed time (s): 156.424363


In [7]:
with timer():
    SCS = df.groupby('Location').apply(clear_sky, info=info, model='simplified_solis')

Elapsed time (s): 157.428710


In [8]:
df['Ineichen'] =  CS.reorder_levels(['Datetime', 'Location'])['ghi']
df['Haurwitz'] = HCS.reorder_levels(['Datetime', 'Location'])['ghi']
df['Solis']    = SCS.reorder_levels(['Datetime', 'Location'])['ghi']

In [9]:
df.head()

Radiation                                GH       GT    ClearSky    Ineichen  \
Datetime                  Location                                             
2010-03-19 14:15:00-10:00 AP1       476.328      NaN  973.629240  840.960548   
                          AP3       382.777      NaN  973.625962  841.600950   
                          AP4       351.610      NaN  973.624679  841.594000   
                          AP5       390.092      NaN  973.630994  841.628209   
                          AP6       353.928  343.313  973.628714  841.615852   

Radiation                             Haurwitz       Solis  
Datetime                  Location                          
2010-03-19 14:15:00-10:00 AP1       871.749955  907.496424  
                          AP3       871.733426  907.477590  
                          AP4       871.726958  907.470220  
                          AP5       871.758798  907.506499  
                          AP6       871.747297  907.493395

In [10]:
df.to_pickle('pvlib.pkl')

For the detect_clearsky function we need to do it per station and day (it does not support unequal differences in index)

In [41]:
with timer():
    cs = pvlib.clearsky.detect_clearsky(df.loc[('2010-03-20', 'AP1'), 'GH'], 
                                        df.loc[('2010-03-20', 'AP1'), 'Ineichen'], 
                                        df.loc[('2010-03-20', 'AP1'), 'GH'].index.get_level_values('Datetime'), 
                                        10)

/home/atorres/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:1238: FutureWarning: Converting timezone-aware DatetimeArray to timezone-naive ndarray with 'datetime64[ns]' dtype. In the future, this will return an ndarray with 'object' dtype where each element is a 'pandas.Timestamp' with the correct 'tz'.
	To accept the future behavior, pass 'dtype=object'.
	To keep the old behavior, pass 'dtype="datetime64[ns]"'.
  a = asanyarray(a)
/home/atorres/.local/lib/python3.6/site-packages/pvlib/clearsky.py:696: RuntimeWarning: invalid value encountered in true_divide
  meas_slope_nstd = np.std(meas_slope, axis=0, ddof=1) / meas_mean
/home/atorres/.local/lib/python3.6/site-packages/pvlib/clearsky.py:721: RuntimeWarning: invalid value encountered in less
  c4 = meas_slope_nstd < var_diff


Elapsed time (s): 122.697683


In [49]:
df.loc[('2010-03-20', 'AP1'), ['GH', 'Ineichen']].loc[cs.to_numpy()]

,Radiation,GH,Ineichen
Datetime,Location,,
2010-03-20 06:58:00-10:00,AP1,44.0718,14.337000
2010-03-20 06:59:00-10:00,AP1,47.3772,16.203300
2010-03-20 07:00:00-10:00,AP1,50.6826,18.179851
2010-03-20 07:01:00-10:00,AP1,54.3553,20.262922
2010-03-20 07:02:00-10:00,AP1,57.6608,22.448634
2010-03-20 07:03:00-10:00,AP1,60.9663,24.733024
2010-03-20 07:04:00-10:00,AP1,64.2717,27.112099
2010-03-20 07:05:00-10:00,AP1,68.3117,29.581878
2010-03-20 07:06:00-10:00,AP1,71.9845,32.138427
